## Extraction de données d'un fichier pour automatiser leur traitement
Le but de cet exercice est de travailler sur l'extraction de données à partir de fichiers. Le sujet étant très vaste, on va ici étudier certaines méthodes et leurs alternatives. Si python permet de faire tout en une seule étape, pour des très gros fichiers (de l'ordre de la centaine de méga-octets), il se peut que cela soit trop lent et qu'il faille lui préférer des outils en ligne de commande, un des plus usités étant `awk`.


On cherche à analyser un fichier généré par un programme de chimie quantique. Celui-ci contient un très grand nombre d'informations et on cherche à en extraire certaines informations. Entre autre, on va chercher à extraire :
- des symétries d'orbitales qui sont listées sur plusieurs lignes consécutives
- la projection d'une orbitale moléculaire sur des orbitales atomiques : cela correspond à des blocs de textes similaires qui se répètent.

Le fichier à analyser s'appelle `exobonus-alizarine.log`, il fait 4018 lignes et a été généré avec le logiciel gaussian. Bien que l'exemple soit spécifique, la succession d'actions est relativement générique et pourra être adaptée à d'autres cas moyennant quelques changements. Le fichier est ici directement au format texte, ce qui simplifie les vérifications. Mais il se peut que le format d'origine soit plus complexe comme un fichier de type hdf ou un fichier binaire.

Globalement, on va avoir une succession de blocs qui vont décrire certains résultats. Puis à l'intérieur d'un bloc, on va pouvoir avoir la répétition de sous-bloc identiques que l'on cherche à analyser.

![pics/struc-files.svg](pics/struc-files.svg)

# Objectifs
On va chercher à :
* récupérer les symétries des orbitales qui sont données dans un bloc de text
* récupérer des coefficients d'orbitales moléculaires, ce qui correspond à une succession de sous-bloc similaires à découper pour ensuite les analyser de manière similaire.

## Analyse du fichier pour récupérer les symétries

Pour les symétries, on va avoir à analyser les lignes 896 à 904 dans le fichier d'exemple. Pour voir leur contenu, il est possible d'utiliser deux utilitaires : soit `sed`, soit `awk`.
 1. Afficher les lignes pour voir ce à quoi elles peuvent ressembler. Pour cela on peut soit :
  * utiliser `sed` : il faut spécifier les lignes que l'on veut étudier avec les commandes `sed -n 'XX,YYp' file` pour afficher les lignes comprises entre `XX` et `YY` du fichier `file`
  * utiliser `awk` : idem, il faut spécifier les lignes avec la commande : `awk 'NR>=XX && NR<=YY' file`
Le résultat donne :
```
 Orbital symmetries:
       Occupied  (A1) (E) (T2) (A) (A) (A) (A) (A) (A) (A) (A) (A)
                 (A2) (A) (B2) (A) (A) (B) (A) (A) (A) (A) (A) (A)
                 (A1) (A) (T1) (A) (A) (A) (A) (A) (A) (A) (A) (A)
                 (A) (A) (A) (A) (B1) (A) (A) (A)
       Virtual   (A2) (A1) (A2) (A) (A) (A) (A) (A) (A) (A) (A) (A)
                 (A) (A) (A) (A) (A) (A) (A) (A) (E) (A) (A) (A)
                 (A) (A) (A) (A) (A) (A) (A) (A) (A) (E1) (A) (A)
 The electronic state is 1-A.
```

On peut ainsi voir que ce bloc est délimité par des mots-clé : 
* `Orbital symmetries:` pour le début du bloc de texte
* ` The electronic state is` pour la fin du bloc de texte
* puis que chaque étiquette de symétrie est données par une succession de une lettre et éventuellement un chiffre entre parenthèses.

Comme stratégie, on va isoler le bloc de texte correspondant. Et on va faire en sorte que le script soit auto-adaptatif pour chaque fichier afin de pouvoir facilement automatiser le traitement. On va donc avoir à faire deux étapes :

a. Isoler le bloc de texte qui nous intéresse

b. Faire du traitement sur le bloc isolé pour extraire les informations pertinentes


### Isoler un bloc de texte
Pour isoler le bloc de texte, on va détecter les lignes qui présentent les mots-clé indiquant le début et la fin du bloc.

1. Rédiger une première fonction `findLineKeywords` qui détecte un ensemble de mots clés présents sur des lignes. Faire en sorte que la fontion accepte une liste de mots-clé et retourne un tableau avec :
   
* un tableau indiquant la position des lignes contenant chacun des mots-clé, par défaut, on retourne la dernière occurence
* le nombre total de lignes
  
De manière générale, on simule l'action de la ligne de command `grep -n` en essayant de trouver des lignes particulières. 

In [1]:
import numpy as np

def findLineKeywords(file,keywords):
    """
    file : filename of the file to analyze
    keywords : list of keywords given as string to find in each lines, only the last occurence of the keyword will be returned
    """
    lineNumber = 1 #We start at line 1
    #we initalize the array that will contain the linenumbers for each keyword, given as integer
    Lines = -np.ones(len(keywords),dtype=int)
    
    with open(file, "r") as f:
        #lecture ligne à ligne du fichier, préférable si jamais le fichier est trop gros pour 
        #être stocké en mémoire
        for lineNo, line in enumerate(f):
            #On cherche chaque mot-clé
            for i,keyword in enumerate(keywords):
                #S'il est trouvé, on place l'indice de la ligne dans la position correspondante du tableau qu'on va retourner
                if line.find(keyword) != -1:
                    Lines[i] = lineNumber
            #On augmente la valeur de la ligne
            lineNumber += 1  
    #On corrige pour le nombre total de ligne qui a été incrémenté une fois de trop à la fin
    return Lines,lineNumber-1
#définition du fichier d'entrée
filename = "files/exobonus-alizarine.log"
#définition des mots-clé à chercher
keywords = [" Orbital symmetries:", " The electronic state is"]
#recherche des mots clé
Positions,LineTotal  = findLineKeywords(filename,keywords)
print("Le fichier contient {} lignes".format(LineTotal))
for i, keyword in enumerate(keywords):
    print("Le mot-clé '{}' est présent à la ligne {:d}".format(keyword,Positions[i]))

Le fichier contient 4018 lignes
Le mot-clé ' Orbital symmetries:' est présent à la ligne 896
Le mot-clé ' The electronic state is' est présent à la ligne 904


### Analyse du bloc
#### Découper le fichier en bloc pour lui appliquer des traitements spécifiques
On va ici découper le bloc de texte qui nous intéresse pour l'isoler. Cela permet de :
* vérifier que le bloc de texte sur lequel on travaille est bien le bon
* pouvoir éventuellement lire uniquement le bloc de texte qui nous intéresse plutôt que de recommencer à lire tout le fichier de départ. C'est intéressant surtout si le fichier de départ est très volumineux.
* On peut bien évidemment supprimer les fichiers annexes créés à la fin si on veut qu'ils soient temporaires.

2. Créer une fonction `cutFile` capable de découper le fichier `infile` initial entre les lignes XX et YY en un plus petit fichier `outfile` à l'aide de la ligne de commande. Pour cela, on pourra utiliser la librairie `subprocess`, en particulier `subprocess.run` qui permet d'exécuter une ligne de commande depuis python comme si on était dans le terminal. Pour utiliser la redirection de la sortie, il est possible d'utiliser les options `shell=True, check=True`. Pour les commandes, on peut utiliser `sed` ou `awk` (voir ci-dessus).


In [2]:
import subprocess

def cutFile(infile,outfile,start,end):
    """
    Découpe un fichier infile à l'aide de sed ou awk pour ne garder que les lignes
    comprises entre les indices start et end (ces lignes sont comprises)
    le tout est placé dans le fichier outfile
    
    infile : nom du fichier d'entrée
    outfile : nom du fichier de sortie
    start : indice de la première ligne, la ligne sera incluse
    end : indice de la dernière ligne, la ligne sera incluse
    
    renvoie :
    le résultat de subprocess
    """
    return subprocess.run(
        "sed -n '{},{}p' {} > '{}'".format(start,end,infile,outfile),         #avec sed
        #"awk 'NR>={} && NR<={}' {} > '{}'".format(start,end,infile,outfile),  #avec awk
        shell=True,
        check=True
    )
#Nom du fichier qui va contenir les symétries, on va prendre la racine du fichier et lui ajouter le suffixe '-syms.txt'
FileSyms = filename.split(".")[0]+'-syms.txt'

#Découpage du fichier, on ajuste pour enlever les lignes qui inutiles du bloc
cutFile(filename,FileSyms,Positions[0]+1,Positions[1]-1)

CompletedProcess(args="sed -n '897,903p' files/exobonus-alizarine.log > 'files/exobonus-alizarine-syms.txt'", returncode=0)

3. Ré-écrire la même fonction en python `cutFilePython` , cela peut permettre de ne pas être dépendant des utilitaires `sed` et `awk` et dépendre du système d'exploitation. Faire attention à ne lire que le minimum de ligne pour être le plus efficace possible si jamais le fichier contient beaucoup de lignes.

In [3]:
def cutFilePython(infile,outfile,start,end):
    """
    Découpe un fichier infile à l'aide de sed ou awk pour ne garder que les lignes
    comprises entre les indices start et end (ces lignes sont comprises)
    le tout est placé dans le fichier outfile
    
    infile : nom du fichier d'entrée
    outfile : nom du fichier de sortie
    start : indice de la première ligne, la ligne sera incluse
    end : indice de la dernière ligne, la ligne sera incluse
    
    renvoie :
    le nom du fichier de sortie
    """
    i=1
    with open(infile, "r") as fin:
        with open(outfile, "w") as fout:
            #Lecture ligne à ligne
            line = fin.readline()
            #On arrête la lecture dès qu'on arrive à la dernière ligne à inclu
            while line and i <= end:
                if i >= start and i <= end:
                    fout.write(line)
                line = fin.readline()
                i+=1
    return outfile
    
cutFilePython(filename,FileSyms,Positions[0]+1,Positions[1]-1)        


'files/exobonus-alizarine-syms.txt'

In [4]:
#Afficher le contenu du fichier ou non
ShowOutputFile = True
if ShowOutputFile:
    with open(FileSyms, "r") as f:
        #lecture ligne à ligne du fichier, préférable si jamais le fichier est trop gros pour 
        #être stocké en mémoire
        for lineNo, line in enumerate(f):
            print(line)

       Occupied  (A1) (E) (T2) (A) (A) (A) (A) (A) (A) (A) (A) (A)

                 (A2) (A) (B2) (A) (A) (B) (A) (A) (A) (A) (A) (A)

                 (A1) (A) (T1) (A) (A) (A) (A) (A) (A) (A) (A) (A)

                 (A) (A) (A) (A) (B1) (A) (A) (A)

       Virtual   (A2) (A1) (A2) (A) (A) (A) (A) (A) (A) (A) (A) (A)

                 (A) (A) (A) (A) (A) (A) (A) (A) (E) (A) (A) (A)

                 (A) (A) (A) (A) (A) (A) (A) (A) (A) (E1) (A) (A)



#### Traitement du fichier pour extraire les données

Maintenant que l'on a découpé le fichier, il faut récupérer toutes les étiquettes de symétrie. On pourrait essayer d'utiliser `np.genfromtxt`, mais ici, le problème est que les données ne sont pas uniformes et le résultat devrait être un tableau unidimensionnel. Ici, les données sont données sous forme bidimensionnelle et le nombre de colonnes n'est pas identique sur toutes les lignes. De plus, toutes les colonnes ne font pas forcément toutes de la même taille, donc on ne peut pas découper en se basant sur des colonnes de largeur fixe. De plus, il y a les mots `Occupied` et `Virtual` uniquement sur certaines lignes ce qui fait que deux lignes ont un comportement particulier. C'est l'occasion idéale ... d'utiliser des expressions régulières qui sont extrêmement souples !

4. À l'aide d'une expression régulière, écrire une fonction capable d'extraire toutes les symétries listées entre parenthèses dans un fichier et qui retourne une liste contenant toutes les étiquettes de symétrie.

In [5]:
import re

def extractSyms(infile):
    """
    Extrait toutes les étiquettes de symétrie contenues entre des parenthèses dans le fichier infile
    retourne une liste contenant toutes les étiquettes de symétrie
    """
    #on créée la liste qui va contenir toutes les étiquettes de symétrie
    data = []
    #on ouvre le fichier contenant la partie contenant les symétries des orbitales occupées.
    regExpSym = re.compile("\((\w*)\)")
    with open(infile, "r") as f:
        for i, line in enumerate(f):
            #on trouve tous les éléments entre parenthèse
            matches = regExpSym.findall(line)
            #on les ajoute unes par unes à la liste des symétries
            data.extend(matches)
        #puis on convertit le tout sous forme de numpy array
        return np.asarray(data)
Syms = extractSyms(FileSyms)
print(Syms)
print(Syms.shape)

['A1' 'E' 'T2' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A2' 'A' 'B2' 'A' 'A'
 'B' 'A' 'A' 'A' 'A' 'A' 'A' 'A1' 'A' 'T1' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'B1' 'A' 'A' 'A' 'A2' 'A1' 'A2' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'E' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'E1' 'A' 'A']
(80,)


# Extraction des données pour extraire des coefficients
Le fichier présente ensuite des données qui donnent la décomposition d'une orbitale moléculaire sur des orbitales atomiques. On va chercher à récupérer les données réparties sur un grand nombre de lignes pour finir par avoir un tableau des coefficients pour toutes les orbitales.

5. Regarder la structure des sous-blocs pour voir le travail à faire. Pour cela, il faut afficher les lignes 922 à 1005. (avec `sed` ou `awk`, cf ci-dessus)

La structure de chaque sous-bloc est celle donnée sur la figure ci-dessous :
* on a 3 lignes d'en tête, puis `N` coefficients
* on a une colonne d'en-tête puis entre 1 et 5 colonnes qui donnent chacune la projection d'une orbitale moléculaire sur les `N` orbitales atomiques.
![pics/struc-sous-bloc.svg](pics/struc-sous-bloc.svg)

6. Utiliser la fonction `findLineKeywords` pour trouver les indices de début et de fin pour le bloc donnant les orbitales moléculaires

*Ici, on multiplie les lectures du fichier de départ, ce n'est pas optimal, mais on pourrait bien évidemment chercher en une seule fois tous les mots-clé et en profiter pour lire le nombre d'orbitales moléculaires à lire.*

In [6]:
#définition des mots-clé à chercher
keywords = [" Molecular Orbital Coefficients:", " Density Matrix:"]
#recherche des mots clé
Positions,LineTotal  = findLineKeywords(filename,keywords)
print("Le fichier contient {} lignes".format(LineTotal))
for i, keyword in enumerate(keywords):
    print("Le mot-clé '{}' est présent à la ligne {:d}".format(keyword,Positions[i]))

Le fichier contient 4018 lignes
Le mot-clé ' Molecular Orbital Coefficients:' est présent à la ligne 922
Le mot-clé ' Density Matrix:' est présent à la ligne 2251


7. Écrire une fonction capable d'aller chercher le nombre `N` d'orbitales atomiques pour savoir le nombre de lignes à lire. Bien faire attention à stocker la valeur sous forme d'entier pour la suite, utiliser une expression régulière.
*Cette information est fournie sous diverses formes dans différentes lignes* (c'est le `80` qui nous intéresse ici) :
```
 There are    80 symmetry adapted cartesian basis functions of A   symmetry.
 There are    80 symmetry adapted basis functions of A   symmetry.
    80 basis functions,   480 primitive gaussians,    80 cartesian basis functions
```
ou
```
 NBasis=    80 RedAO= F EigKep=  0.00D+00  NBF=    80
 NBsUse=    80 1.00D-04 EigRej=  0.00D+00 NBFU=    80
```

In [7]:
findOAS = re.compile(" There are\s*(\d*)\ssymmetry adapted basis functions")
with open(filename, "r") as f:
    for i, line in enumerate(f):
        #On cherche via l'expression régulière
        orbs = findOAS.match(line)
        #Si l'expression régulière a trouvé un correspondance, on la stocke
        if orbs:
            NbOA = int(orbs.group(1))
print(NbOA)

80


On va maintenant découper le fichier en plus petits fichiers pour ensuite pouvoir automatiser le traitement de chacun des sous fichiers. On pourrait tout faire en une seule étape, mais cela complexifierait le code dans son ensemble. Ici, je préfère répéter une succession d'actions simples plutôt qu'un unique traitement plus complexe. Pour cela, on va ré-utiliser la fonction déjà créée auparavant pour découper les fichiers.


8. Découper le fichier de départ en chacun des sous-blocs, on pourra pour cela utiliser la fonction `cutFile` ainsi que les résultats de la question 6. 

In [8]:
#On va découper tant que c'est nécessaire et qu'on est dans la bonne section

lineStartCoeffs, lineEndCoeffs = Positions
#Ici, on ajoute 1 car la détection se faisait sur la ligne qui ouvre la section
startLine = lineStartCoeffs+1
#Numéro de fichier de sortie que l'on va créer
i = 0
#On vérifie qu'on reste dans la bonne section pour découper le fichier
while startLine < lineEndCoeffs:
    #Découpage du fichier
    #Ici, on définit la valeur de fin de la sélection
    endLine = startLine + NbOA+2
    #pour les fichiers de sortie, on va ajouter des suffixes
    cutFile(filename,'{}-orbs-bloc-{:03d}.txt'.format(filename.split(".")[0],i),startLine,endLine)
    #On incrémente startline pour la prochaine itération
    startLine += NbOA+3
    #on incrémente le numéro de fichier.
    i += 1
#On va stocker le nombre de fichier qu'il va falloir traiter.
EndFile = i

Une fois les fichiers découpés, il va être possible d'automatiser le traitement. On peut être plus ou moins souple pour automatiser les choses. Ici, vu que le fichier est relativement bien formaté, on va pouvoir adapter différentes méthodes de traitement. Il serait également possible d'utiliser `awk` pour préformater les données, mais ici, on va appliquer différentes méthodes pour faire des choses plus ou moins compliquées.

## Aller au plus simple avec np.genfromtxt pour lire les coefficients

Si on suppose que les données sont très bien formatées, il est possible de faire en sorte de donner des positions de début et de fin des colonnes. Pour cela, on va spécifier la longueur des colonnes. À la fin, il se peut que le nombre de colonnes ne soit pas égal à 5 dans le cas le plus général. Il faut donc enlever toutes les colonnes qui ne contiennent que des `nan` : il y aura forcément la première qui contient les labels des orbitales atomiques, et peut être des colonnes manquantes à la fin. Il reste ensuite à concaténer tous les coefficients  correctement.

9. Utiliser `np.genfromtxt`, utiliser l'option  `delimiter` avec une suite d'entier pour indiquer la longueur des colonnes à lire `[23,10,10,10,10,10]`. Les trois premières lignes n'étant "pas intéressantes" ici, il est possible de ne pas les lire. De même, la première colonne n'est pas intéressante pour tout de suite. *Il aurait aussi été possible de faire un parser avec des expressions régulières, mais cela aurait été plus long à programmer et à débugguer, j'ai préféré ici privilégier une fonction toute faite relativement robuste*

In [9]:
#On créée une liste qui va contenir les données extraites
CoeffMos = []
for i in range(EndFile):
    
    tempData = np.genfromtxt('{}-orbs-bloc-{:03d}.txt'.format(filename.split(".")[0],i),skip_header = 3, delimiter = [23,10,10,10,10,10],dtype="float32",usecols=range(1,6))
    #On enlève les colonnes qui ne contiennent que des nan si jamais il y en a à la fin.
    tempData = tempData[:,~np.all(np.isnan(tempData), axis=0)]
    CoeffMos.append(tempData)
CoeffMos = np.concatenate(CoeffMos,axis=1)
print(CoeffMos.shape)

(80, 80)


## Lire les étiquettes des orbitales atomiques
Pour les étiquettes des orbitales atomiques, on a une succession de 4 labels :
* un numéro unique associé à l'orbitale atomique qui va de 1 à 80
* le numéro de l'atome qui porte l'orbitale atomique
* le type d'atome qui porte l'orbitale atomique (carbone, hydrogène, etc)
* le type d'orbitale correspondant : 1S, 2PX, etc


10. Lire les étiquettes dans un des fichiers correspondant aux différents sous-blocs avec `np.genfromtxt`. On pourra utiliser `delimiter` ou tout autre  moyen. On peut également chercher à appliquer un filtre via l'option `converters` pour supprimer tous les espaces inutiles en début et en fin de chaîne de caractère.

In [10]:
OALabels = np.genfromtxt('{}-orbs-bloc-{:03d}.txt'.format(filename.split(".")[0],0),converters={i: lambda x: x.decode().strip() for i in range(4)}, skip_header = 3, delimiter = [4,3,4,5], dtype="str")

print(OALabels[:5,:])

[['1' '1' 'C' '1S']
 ['2' '' '' '2PX']
 ['3' '' '' '2PY']
 ['4' '' '' '2PZ']
 ['5' '2' 'C' '1S']]


On a récupéré les colonnes, mais il y a des données manquantes, par exemple, pour l'orbitale 3, il manque le numéro d'atome et le type d'atome. 

11. Écrire une portion de code capable de compléter les colonnes vides avec les dernières valeurs non vides dans la ligne au dessus.

In [11]:
atomNumber = ''
atomType = ''
for line in range(OALabels.shape[0]):
    #On regarde si le numéro d'atome est vide ou non, 
    #s'il est non vide on l'actualise, 
    #s'il est vide, on met la dernière valeur rencontrée
    #On fait de même pour préciser le type d'atome.
    if OALabels[line,1] != '':
        atomNumber = OALabels[line,1]
    else :
        OALabels[line,1] = atomNumber
    if OALabels[line,2] != '':
        atomType = OALabels[line,2]
    else :
        OALabels[line,2] = atomType    
print(OALabels[:5,:])

[['1' '1' 'C' '1S']
 ['2' '1' 'C' '2PX']
 ['3' '1' 'C' '2PY']
 ['4' '1' 'C' '2PZ']
 ['5' '2' 'C' '1S']]


## Un exemple d'utilisation du travail effectué

On peut maintenant faire des choses sympatiques comme afficher tous les coefficients importants pour chaque orbitale moléculaire.



In [12]:
#On va créer un label unique
OALabelsLong = np.array([" ".join(r) for r in OALabels])
#On va afficher pour quelques orbitale les coefficients plus grands qu'une valeur seuil
displayThres = 0.2
for i in [0,10,15,20]:
    print('Coefficients prépondérants pour l\'OA {}'.format(i+1))
    mask = np.abs(CoeffMos[:,i]) > displayThres
    Essai = np.vstack((OALabelsLong[mask],CoeffMos[mask,i]))
    np.set_printoptions(precision=5)
    print(Essai.T)

Coefficients prépondérants pour l'OA 1
[['17 5 C 1S' '0.29368']
 ['25 7 C 1S' '0.29895']
 ['29 8 O 1S' '0.28772']
 ['37 10 C 1S' '0.37943']
 ['61 16 C 1S' '0.27324']
 ['65 17 O 1S' '0.2398']
 ['69 18 O 1S' '0.41201']
 ['80 26 H 1S' '0.20824']]
Coefficients prépondérants pour l'OA 11
[['33 9 C 1S' '-0.22803']
 ['42 11 C 2PX' '-0.27102']
 ['49 13 C 1S' '0.21011']
 ['57 15 C 1S' '0.38462']
 ['63 16 C 2PY' '0.29187']
 ['78 24 H 1S' '0.27221']]
Coefficients prépondérants pour l'OA 16
[['10 3 C 2PX' '-0.258']
 ['18 5 C 2PX' '-0.24288']
 ['34 9 C 2PX' '0.23814']
 ['50 13 C 2PX' '-0.23663']
 ['54 14 C 2PX' '-0.20575']
 ['58 15 C 2PX' '-0.25274']
 ['66 17 O 2PX' '0.34275']]
Coefficients prépondérants pour l'OA 21
[['9 3 C 1S' '0.20383']
 ['31 8 O 2PY' '0.20571']
 ['47 12 O 2PY' '-0.23035']
 ['50 13 C 2PX' '-0.24868']
 ['58 15 C 2PX' '0.26112']
 ['71 18 O 2PY' '-0.21346']
 ['76 22 H 1S' '-0.22139']
 ['78 24 H 1S' '-0.21219']]


### Quelques alternatives

#### Utilisation native de grep ou awk pour détecter les lignes
Pour trouver les lignes contenant les mots-clés, on peut utiliser `grep -n` et pour compter les lignes, on peut utiliser la commande `grep -c ^ file`

In [13]:
def findLineKeywordsGrep(file,keywords):
    Lines = -np.ones(len(keywords),dtype=int)
    for i,keyword in enumerate(keywords):
        #on utilise l'option stdout pour récupérer la sortie puis decode car la sortie sera donnée sous forme binaire
        result = subprocess.run("grep -n \"{}\" {}".format(keyword,file) , stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8')
        #alternative avec awk : awk "/ Orbital symmetries:/ {print NR}" files/exobonus-alizarine.log
        
        #On découpe le résultat en lignes,
        #on récupère l'avant dernière (la dernière est une ligne vide) 
        # puis on récupère le numéro de ligne qui est donné avant les ":"
        Lines[i] = int(result.split("\n")[-2].split(":")[0])
    #On compte les lignes        
    lineNumber = int(subprocess.run("grep -c ^ {}".format(file) , stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8'))
    return Lines,lineNumber

#définition du fichier d'entrée
filename = "files/exobonus-alizarine.log"
#définition des mots-clé à chercher
keywords = [" Orbital symmetries:", " The electronic state is"]
#recherche des mots clé
Positions,LineTotal  = findLineKeywordsGrep(filename,keywords)
print("Le fichier contient {} lignes".format(LineTotal))
for i, keyword in enumerate(keywords):
    print("Le mot-clé '{}' est présent à la ligne {:d}".format(keyword,Positions[i]))

Le fichier contient 4018 lignes
Le mot-clé ' Orbital symmetries:' est présent à la ligne 896
Le mot-clé ' The electronic state is' est présent à la ligne 904


In [14]:
#alternative avec awk plutôt que grep :
def findLineKeywordsAwk(file,keywords):
    Lines = -np.ones(len(keywords),dtype=int)
    for i,keyword in enumerate(keywords):
        #on utilise l'option stdout pour récupérer la sortie puis decode car la sortie sera donnée sous forme binaire
        #attention aux "{" qui sont des caractères spéciaux et doivent être transformés en "{{"
        #dans le terminal, on utiliserait awk "/keyword/ {print NR}" file
        result = subprocess.run("awk \"/{}/ {{print NR}}\" {}".format(keyword,file) , stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8')
        #on récupère l'avant dernière (la dernière est une ligne vide) 
        Lines[i] = int(result.split("\n")[-2])
    #On compte les lignes
    lineNumber = int(subprocess.run("awk 'END {{print NR}}' {}".format(file) , stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8'))
    return Lines,lineNumber    
#recherche des mots clé
Positions,LineTotal  = findLineKeywordsAwk(filename,keywords)
print("Le fichier contient {} lignes".format(LineTotal))
for i, keyword in enumerate(keywords):
    print("Le mot-clé '{}' est présent à la ligne {:d}".format(keyword,Positions[i]))    

Le fichier contient 4018 lignes
Le mot-clé ' Orbital symmetries:' est présent à la ligne 896
Le mot-clé ' The electronic state is' est présent à la ligne 904


## Une autre méthode sans découper en sous-fichiers pour lire les coefficients

On peut également faire un essai en faisant un unique np.genfromtxt depuis le fichier de départ, mais cela demande plus de réflexion. En effet, on doit inclure les lignes d'en tête, puis les supprimer et faire un reformatage intelligent du tableau avec une combinaison de `split` et `stack`. Par contre, cela donne assez simplement accès aux valeurs propres associées à chaque orbitale.

In [34]:
def removeNanCols(data):
    """
    Function to remove the columns containing only Nans
    """
    return data[:,~np.all(np.isnan(data.astype('float32',casting='unsafe')), axis=0)]

#définition des mots-clé à chercher
keywords = [" Molecular Orbital Coefficients:", " Density Matrix:"]
#recherche des mots clé
Positions,LineTotal  = findLineKeywords(filename,keywords)
print("Le fichier contient {} lignes".format(LineTotal))
for i, keyword in enumerate(keywords):
    print("Le mot-clé '{}' est présent à la ligne {:d}".format(keyword,Positions[i]))
lineStartCoeffs, lineEndCoeffs = Positions
    

#On lit toutes les lignes de la section en une seule fois pour obtenir 6 colonnes
tryData = np.genfromtxt(filename, skip_header = lineStartCoeffs, skip_footer = LineTotal - lineEndCoeffs+1, delimiter = [23,10,10,10,10,10], dtype="float32")

#Il faut maintenant découper le tableau en portions de 5 colonnes, pour cela, il va falloir compter combien il y en a :
NbCuts = np.ceil(NbOA/5)

#On fait un découpage dans le sens de la hauteur avec split, axis=0 et on recolle les morceaux dans le sens de la largeur avec stack, axis=1
test=np.stack(np.split(tryData,int(NbCuts),axis=0),axis=1)

#Pour récupérer l'indice de l'orbitale, un peu compliqué pour ce que pourrait faire un simple np.arange
Numbers =  test[0,:]
Numbers = Numbers[~(np.isnan(Numbers))]
#print(Numbers)
#Pour récupérer les valeurs propres associées à chaque OM
Eigenvalues =  test[2,:]
Eigenvalues = Eigenvalues[~(np.isnan(Eigenvalues))]
print(Eigenvalues)

#Pour récupérer les coefficients
#On enlève les lignes d'en-tête qui contiennent le numéro de l'OM, le fait qu'elle soit occupée ou vacante et la valeur propre associée
test = test[3:,:]
#On enlève les colonnes qui seraient vides si jamais il y en a, ici, vu que le nombre d'orbitales est multiple de 5, il ne se passe rien
test = removeNanCols(test)
print(test.shape)
#Un petit test pour vérifier que l'on a bien obtenu les mêmes choses
print(np.array_equal(CoeffMos,test)) 

Le fichier contient 4018 lignes
Le mot-clé ' Molecular Orbital Coefficients:' est présent à la ligne 922
Le mot-clé ' Density Matrix:' est présent à la ligne 2251
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71. 72.
 73. 74. 75. 76. 77. 78. 79. 80.]
[-1.76931e+00 -1.68031e+00 -1.64569e+00 -1.59050e+00 -1.52270e+00
 -1.50103e+00 -1.34947e+00 -1.23744e+00 -1.18460e+00 -1.14541e+00
 -1.04123e+00 -1.00259e+00 -9.92020e-01 -9.19970e-01 -8.87780e-01
 -8.40510e-01 -7.75960e-01 -7.30530e-01 -7.23030e-01 -6.87990e-01
 -6.63050e-01 -6.30770e-01 -6.04300e-01 -6.01180e-01 -5.89710e-01
 -5.82820e-01 -5.70110e-01 -5.58990e-01 -5.55850e-01 -5.23050e-01
 -5.09180e-01 -4.98870e-01 -4.89440e-01 -4.83460e-01 -4.81700e-01
 -4.75000e-01 -4.67950e-01 -4.39660e-01 -3.74210e